# Project -- Movie Recommendation System

In [1]:
# importing all libraries
import numpy as np
import pandas as pd

import nltk
from nltk.stem.porter import PorterStemmer; # for stemming

from sklearn.feature_extraction.text import CountVectorizer # for Vectorization
from sklearn.metrics.pairwise import cosine_similarity # calculating cosine distances b/w every movies
import pickle # to save the model

**Data colection and pre-processing**

In [2]:
movies=pd.read_csv('movies.csv')

In [3]:
movies.shape # size of the data frame

(4803, 24)

In [4]:
# checking some samples of the dataframe
movies.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [5]:
movies=pd.read_csv('movies.csv',index_col='index') # we use 'index' col as index

In [6]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
index,,,,,,,,,,,,,,,,,,,,,
0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [7]:
# checking attributes of the dataframe
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4775 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4391 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [8]:
# filtering out only useful columns
movies=movies[['id','title','overview','genres','keywords','cast','director']]

In [9]:
# checking missing values for each column
movies.isnull().sum()

id            0
title         0
overview      3
genres       28
keywords    412
cast         43
director     30
dtype: int64

In [10]:
# filling up missing values
movies.fillna('missing',inplace=True)

In [11]:
movies.isnull().sum()

id          0
title       0
overview    0
genres      0
keywords    0
cast        0
director    0
dtype: int64

In [12]:
# removing spaces for each director names

movies.loc[movies['director'].notnull(), 'director'] = movies.loc[movies['director'].notnull(), 'director'].astype(str).str.replace(' ','')
movies['director'] = movies['director'].replace(np.nan, '', regex=True) # replacing nan value 

In [13]:
movies['director']

index
0           JamesCameron
1          GoreVerbinski
2              SamMendes
3       ChristopherNolan
4          AndrewStanton
              ...       
4798     RobertRodriguez
4799         EdwardBurns
4800          ScottSmith
4801          DanielHsia
4802     BrianHerzlinger
Name: director, Length: 4803, dtype: object

In [14]:
# converting colums into lists

movies['overview']=movies['overview'].apply(lambda x:x.split())
movies['genres']=movies['genres'].apply(lambda x:x.split())
movies['cast']=movies['cast'].apply(lambda x:x.split())
movies['director']=movies['director'].apply(lambda x:x.split())
movies['keywords']=movies['keywords'].apply(lambda x:x.split())

In [15]:
movies.head()

,id,title,overview,genres,keywords,cast,director
index,,,,,,,
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science, Fiction]","[culture, clash, future, space, war, space, co...","[Sam, Worthington, Zoe, Saldana, Sigourney, We...",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug, abuse, exotic, island, east, ind...","[Johnny, Depp, Orlando, Bloom, Keira, Knightle...",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based, on, novel, secret, agent, sequel,...","[Daniel, Craig, Christoph, Waltz, L\u00e9a, Se...",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc, comics, crime, fighter, terrorist, secret...","[Christian, Bale, Michael, Caine, Gary, Oldman...",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science, Fiction]","[based, on, novel, mars, medallion, space, tra...","[Taylor, Kitsch, Lynn, Collins, Samantha, Mort...",[AndrewStanton]


In [16]:
# concating overview,genres,keywords,cast,director into a single column named 'tags'
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['director']

In [17]:
movies=movies[['id','title','tags']] # final df we required 

In [18]:
movies.head()

,id,title,tags
index,,,
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [19]:
movies['tags']=movies['tags'].apply(lambda x:" ".join(x)) # converting tag lists into String

In [20]:
movies['tags']=movies['tags'].apply(lambda x:x.lower()) # converting into lower case

In [21]:
# final data-frame
movies.head()

,id,title,tags
index,,,
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


**Stemming 'tags' column** (converting all words to it's root word)

In [22]:
ps=PorterStemmer()

In [23]:
# for each text in tags apply stemming on it
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [24]:
movies['tags']=movies['tags'].apply(stem)

In [25]:
movies.head() # after stemming dataframe

,id,title,tags
index,,,
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."


**Vectorization** (counting most common 10000 words among all tags)

In [26]:
cv=CountVectorizer(max_features=10000,stop_words='english')

In [27]:
vectors=cv.fit_transform(movies['tags']).toarray()

In [28]:
vectors.shape

(4803, 10000)

**Calculating Similarity** (calculating angular distance between two vectors of movies) 

In [29]:
similarity=cosine_similarity(vectors)

In [30]:
def recommend(movie):
    ind=movies[movies['title']==movie].index[0] # fetching the selected movie index
    distances=similarity[ind] # calculating similarity with other movies
    
    # sort the distances to find the most similar 10 movies 
    movie_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:11]
    for i in movie_list:
        print(movies.iloc[i[0]].title)

**Checking Outputs**

In [31]:
recommend('Harry Potter and the Half-Blood Prince')

Harry Potter and the Goblet of Fire
Harry Potter and the Order of the Phoenix
Harry Potter and the Philosopher's Stone
Harry Potter and the Chamber of Secrets
Harry Potter and the Prisoner of Azkaban
The NeverEnding Story
The Adventurer: The Curse of the Midas Box
There Will Be Blood
Stardust
Spirited Away


**Saving the model using pickle**

In [32]:
pickle.dump(movies,open('movies.pkl','wb'))

In [33]:
pickle.dump(similarity,open('similarity.pkl','wb'))